# Show zero data
Many target data have zero value.
Perhaps the product with zero values continuously from the beginning is before sale.
It's better to remove these as they interfere with learning.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Calculate the cumulative sum.

In [ ]:
idcol = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']
df = pd.read_csv("/kaggle/input/m5-forecasting-accuracy/sales_train_evaluation.csv")
tmp = df.set_index(idcol).stack()
tmp = tmp.reset_index()
tmp.rename(columns={"level_6": "d", 0: "target"}, inplace=True)
tmp["before_release"] = tmp[["id", "target"]].groupby("id")["target"].cumsum()

## View before release days.

In [ ]:
zero_count = tmp[["id", "before_release"]][tmp["before_release"]==0].groupby("id").count()
zero_count.hist(bins=100)

## Histogram of products that have been released for over a year.

In [ ]:
zero_count[zero_count["before_release"] > 365].hist(bins=100)

In [ ]:
zero_ids = zero_count[zero_count["before_release"] > 365].index.values

In [ ]:
print("Before remove... {}".format(tmp["target"].count()))
removed = tmp
removed.loc[(removed["before_release"] == 0) & (removed["id"].isin(zero_ids)), "target"] = np.nan
print("After remove...  {}".format(removed["target"].count()))

This process will produce a better score.